# RW - API management functions

These functions will cover the functions need to manage assets within the RW-API and will be base on the code develop for [OW migration from staging to db](https://github.com/resource-watch/notebooks/blob/develop/ResourceWatch/example_migrate_script.ipynb)

In [13]:
%run './pydantic_data_classes_rw.ipynb'

In [14]:
from requests_toolbelt import sessions
from requests_toolbelt.auth.handler import AuthHandler
from requests.adapters import HTTPAdapter
from requests import Session
from requests.packages.urllib3.util.retry import Retry
from datetime import datetime

import types
import json
import logging
import getpass

from pydantic import parse_obj_as

logger = logging.getLogger()
logger.setLevel(logging.INFO)

print('\n'.join(f'{module.__name__}=={getattr(module, "__version__", None)}' for  i, module in globals().items() if isinstance(module, types.ModuleType)))

builtins==None
builtins==None
requests_toolbelt.sessions==None
types==None
json==2.0.9
logging==0.5.1.2
getpass==None


In [15]:
class bcolors:
    '''
    Color definitions for printing
    {bcolors.color}{string}{bcolors.ENDC}
    '''
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    ENDC = '\033[0m'


In [16]:
DEFAULT_TIMEOUT = 60 # seconds

class TimeoutHTTPAdapter(HTTPAdapter):
    '''
    
    '''
    def __init__(self, *args, **kwargs):
        self.timeout = DEFAULT_TIMEOUT
        if "timeout" in kwargs:
            self.timeout = kwargs["timeout"]
            del kwargs["timeout"]
        super().__init__(*args, **kwargs)

    def send(self, request, **kwargs):
        timeout = kwargs.get("timeout")
        if timeout is None:
            kwargs["timeout"] = self.timeout
        return super().send(request, **kwargs)

In [17]:
def auth(session: Session):
    '''
    Authenticates into the RW API
    '''
    print(f'You are login into {bcolors.HEADER}{bcolors.BOLD}RW api{bcolors.ENDC}')
    
    headers = {'Content-Type': 'application/json'}
    payload = json.dumps({ 'email': f'{input(f"Email: ")}',
                           'password': f'{getpass.getpass(prompt="Password: ")}'})
    response = session.post('/auth/login',  headers = headers,  data = payload)
    
    print(f'{bcolors.OKGREEN}Successfully logged into RW api{bcolors.ENDC}')
    
    return response.json().get('data').get('token')

In [18]:
def rwApiSession(serverUrl: str = "https://api.resourcewatch.org"):
    '''
    Creates a sesion object for making calls to the RW api.
    '''
    retries = Retry(total=3, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
    
    assert_status_hook = lambda response, *args, **kwargs: response.raise_for_status()

    httpsSession = sessions.BaseUrlSession(base_url = serverUrl )
    
    httpsSession.mount("https://", TimeoutHTTPAdapter(max_retries=retries))
    httpsSession.hooks["response"] = [assert_status_hook]
    
    token = auth(httpsSession)
    
    httpsSession.headers['Authorization'] = f'Bearer {token}'
    
    return httpsSession

In [19]:
def copyAssetBody(asset: dict, excludeList: List[str] = ['createdAt', 'updatedAt','clonedHost',
                                                         'errorMessage', 'taskId', 'status', 'sources',
                                                         'userId', 'slug', 'dataset', 'layer', 
                                                         'widget', 'metadata', 'vocabulary']):
    '''
    Copy a body dict to a new dict excluding some keys or not defined values.
    '''
    response = {}
    response.update(asset)
    
    for key, value in asset.items():
        if (key in excludeList or value is None or (type(value) == dict and len(value) == 0) ):
            response.pop(key, None)
    
    if 'provider' in response.keys() and response['provider'] =='cartodb':
        response.pop('tableName', None)
    
    return response

In [20]:
def getApiResource(session: Session, url: str, payload: dict, 
                resourceType: Union[List[APIRESOURCE], APIRESOURCE],
                pagination: bool = False
                ) -> Union[List[APIRESOURCE], APIRESOURCE]:
    '''
    Gets an API resource from the selected env or from the 
    constrained dataset list for our needs we will ignore 
    other include than layer.

    Parameters
    ----------
    session : Session object, Session object for making 
              requests to the RW api
    url : str, url to the api resource
    payload : dict, payload to be sent to the api
    resourceType : Union[List[APIRESOURCE], APIRESOURCE], 
                type of resource to be returned
    pagination : bool, if True returns a list of resources

    Returns
    -------
    Union[List[APIRESOURCE], APIRESOURCE], list of resources
    or a single resource

    '''
    response: Union[List[dict], dict]
    
    data = session.get(url, params=payload).json()
    
    response = data.get('data').copy()
    
    # If pagination is needed we will access the data chunk by chunk
    if data.get('meta',{}).get('total-pages',0) > 1 and pagination:
        for n in range(2, data['meta']['total-pages'] + 1):
            payload['page[number]'] = n
            response.extend(session.get(url, params=payload)
                                   .json()
                                   .get('data'))
    
    return parse_obj_as(resourceType, response)
    

In [21]:
def getAssetList(session: Session, application: List[APPLICATION] = ['rw'], 
                 env: ENVIRONMENT = 'production', 
                 includes: List[INCLUDES] = 'layer') -> List[Dataset]:
    '''
    Gets a list of assets from the selected env or from the constrained dataset list
    For our needs we will ignore other includer than layer.
    '''
    url = '/v1/dataset'
    payload={
        'env': env,
        'application':application,
        'status':'saved',
        'includes': includes,
        'page[size]':100
    }
    
    return getApiResource(session, url, payload, List[Dataset])

In [22]:
def getLayerList(session: Session):
    '''
    Gets a list of assets from the selected env or from the layers.
    '''
    url = '/v1/layer'
    payload={
        'env':'production',
        'application':'rw',
        'status':'saved',
        'page[size]':100
    }
    
    return getApiResource(session, url, payload, List[Layer])

In [23]:
def getWidgetList(session: Session):
    '''
    Gets a list of assets from the selected env or from the layers.
    '''
    url = '/v1/widget'
    payload={
        'env':'production',
        'application':'rw',
        'status':'saved',
        'page[size]':100
    }
    
    return getApiResource(session, url, payload, List[Widget])

In [24]:
def backupAssets(session: Session, env: ENVIRONMENT = 'prod'):
    '''
    save a backup of production data just in case we need to recreate it again
    '''
    data = getAssetList(session)

    with open(f'RW_{env}_backup_{datetime.now().strftime("%Y%m%d-%H%M%S")}.json', 'w') as outfile:
        json.dump(data, outfile)

In [25]:
def createOrUpdatedataset(session: Session, dataset: Dataset, toEnv: ENVIRONMENT = 'staging', 
                          destinationDatasetId: Optional[Union[UUID, None]] = None):
    '''
    Copy the dataset from one env to the other
    '''
    if dataset.get('type')!='dataset':
        return None
    
    baseDatasetUrl = f'/v1/dataset'

    body = {'dataset': copyAssetBody(dataset.get('attributes'))}
    
    logger.debug(body)
    
    # Upsert operation
    if destinationDatasetId: 
        return session.patch(f'{baseDatasetUrl}/{destinationDatasetId}',json = body['dataset']).json()
    else:
        return session.post(baseDatasetUrl, body['dataset']).json()

In [26]:
def createOrUpdateLayer(session: Session, datasetId: UUID, layer: Layer, 
                        toEnv: ENVIRONMENT = 'staging', destinationLayerId: Optional[Union[UUID, None]] = None):
    '''
    Copy the layer from one env to the other
    '''
    if layer.get('type')!='layer':
        return None
    baseDatasetUrl = f'/v1/dataset/{datasetId}/layer'

    body = copyAssetBody(layer.get('attributes'))
    
    logger.debug(body)
    
    # Upsert operation
    if destinationLayerId: 
        return session.patch(f'{baseDatasetUrl}/{destinationLayerId}', json = body['dataset']).json()
    else:
        return session.post(baseDatasetUrl, body['dataset']).json()